# DBM Mask R-CNN Code

*Upload the unzipped version of the DBM Mask RCNN folder into your Google Drive's "My Drive" before running the code for the first time, or if any of the files inside the folder has been updated*

### Importing the required libraries for Weights & Biases integration and for Mask R-CNN code

In [ ]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
!pip install tensorflow==1.15.0rc2
import tensorflow as tf
!pip install keras==2.2.5
import keras.backend

     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 512kB 43.2MB/s 
     |████████████████████████████████| 3.8MB 50.0MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=f0bc0490f9bc011eeb13c0fffd928ce943c69fe931e0d0d0d96806f4fcb42c99
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

Using TensorFlow backend.


### Installing and logging into Weights & Biases account

In [ ]:
%%capture
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

!wandb login

wandb: WARNING Keras version 2.2.5 is not fully supported. Required keras >= 2.4.0


wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Mounting drive to upload files from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


### Uploading DBM dataset file (with images and XML files) and visualize, utils & model Python files

In [ ]:
!unzip "/gdrive/My Drive/DBM Mask RCNN/DBM.zip"
!cp "/gdrive/My Drive/DBM Mask RCNN/visualize.py" "visualize.py"
!cp "/gdrive/My Drive/DBM Mask RCNN/utils.py" "utils.py"
!cp "/gdrive/My Drive/DBM Mask RCNN/model.py" "model.py"

Archive:  /gdrive/My Drive/DBM Mask RCNN/DBM.zip
   creating: DBM/annots/
  inflating: DBM/annots/00001.xml    
  inflating: DBM/annots/00002.xml    
  inflating: DBM/annots/00003.xml    
  inflating: DBM/annots/00004.xml    
  inflating: DBM/annots/00005.xml    
  inflating: DBM/annots/00006.xml    
  inflating: DBM/annots/00007.xml    
  inflating: DBM/annots/00008.xml    
  inflating: DBM/annots/00009.xml    
  inflating: DBM/annots/00010.xml    
  inflating: DBM/annots/00011.xml    
  inflating: DBM/annots/00012.xml    
  inflating: DBM/annots/00013.xml    
  inflating: DBM/annots/00014.xml    
  inflating: DBM/annots/00015.xml    
  inflating: DBM/annots/00016.xml    
  inflating: DBM/annots/00017.xml    
  inflating: DBM/annots/00018.xml    
  inflating: DBM/annots/00019.xml    
  inflating: DBM/annots/00020.xml    
  inflating: DBM/annots/00021.xml    
  inflating: DBM/annots/00022.xml    
  inflating: DBM/annots/00023.xml    
  inflating: DBM/annots/00024.xml    
  inflating: D

### Importing the required libraries for Mask R-CNN code

In [ ]:
import xml.etree
from numpy import zeros, asarray

!pip install mrcnn
import mrcnn
import mrcnn.config
import visualize
import utils
import model

     |████████████████████████████████| 61kB 7.9MB/s 
  Created wheel for mrcnn: filename=mrcnn-0.2-cp37-none-any.whl size=54930 sha256=0c6d43fb406bb9cb15a52c29678bfb9c67db74f702be315053e32b3d6bf5f49a
  Stored in directory: /root/.cache/pip/wheels/11/ed/28/e550ddc897c04c336b923eae4eb35c9aae993d20ce39d9cc40
Successfully built mrcnn


### DBM Configuration

In [ ]:
class DBMConfig(mrcnn.config.Config): 
    NAME = "DBM_cfg"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    NUM_CLASSES = 2 #(background/non-DBM + DBM)

    STEPS_PER_EPOCH = 20

    PRE_NMS_LIMIT = 6000
  
DBM_config = DBMConfig()
DBM_config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

### DBM Dataset

In [ ]:
class DBMDataset(utils.Dataset):
  
  def load_dataset(self, dataset_dir, is_train = True):
    self.add_class("dataset", 1, "DBM")
    
    images_dir = dataset_dir + '/images/'
    annotations_dir = dataset_dir + '/annots/'
    
    for filename in os.listdir(images_dir):
      image_id = filename[:-4] # removes .png from the name of the file

      # skip all images before 80 if we are building the train set
      if is_train and int(image_id) <= 80:
        continue

      # skip all images after 80 if we are building the test/val set
      if not is_train and int(image_id) > 80:
        continue
      
      img_path = images_dir + filename
      ann_path = annotations_dir + image_id + '.xml'
      
      self.add_image('dataset', image_id = image_id, path = img_path, annotation = ann_path)

  def extract_boxes(self, filename):
    tree = xml.etree.ElementTree.parse(filename)
    root = tree.getroot()
    
    boxes = list()
    for box in root.findall('.//bndbox'):
      xmin = int(box.find('xmin').text)
      ymin = int(box.find('ymin').text)
      xmax = int(box.find('xmax').text)
      ymax = int(box.find('ymax').text)
      coors = [xmin, ymin, xmax, ymax]
      boxes.append(coors)
    
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height
    
  def load_mask(self, image_id):
    info = self.image_info[image_id]
    path = info['annotation']
    boxes, w, h = self.extract_boxes(path)
    masks = zeros([h, w, len(boxes)], dtype = 'uint8')

    class_ids = list()
    for i in range(len(boxes)):
      box = boxes[i]
      row_s, row_e = box[1], box[3]
      col_s, col_e = box[0], box[2]
      masks[row_s:row_e, col_s:col_e, i] = 1
      class_ids.append(self.class_names.index('DBM'))
    return masks, asarray(class_ids, dtype = 'int32')

### Prepare training and validation sets

In [ ]:
train_set = DBMDataset()
train_set.load_dataset(dataset_dir = 'DBM', is_train = True)
train_set.prepare()

valid_dataset = DBMDataset()
valid_dataset.load_dataset(dataset_dir = 'DBM', is_train = False)
valid_dataset.prepare()

### Initializing and configuring Weights & Biases with the training run

In [ ]:
config = wandb.config

In [ ]:
'''
run = wandb.init(project='my-keras-integration',
                 config={  # and include hyperparameters and metadata
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 1,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "DBM"
                 })
config = wandb.config  # We'll use this to configure our experiment
'''

'\nrun = wandb.init(project=\'my-keras-integration\',\n                 config={  # and include hyperparameters and metadata\n                     "learning_rate": 0.001,\n                     "epochs": 5,\n                     "batch_size": 1,\n                     "loss_function": "sparse_categorical_crossentropy",\n                     "architecture": "CNN",\n                     "dataset": "DBM"\n                 })\nconfig = wandb.config  # We\'ll use this to configure our experiment\n'

In [ ]:
'''
run = wandb.init(project='hyperparameter-run',
                 config=DBM_config)
config = wandb.config  # We'll use this to configure our experiment
'''

"\nrun = wandb.init(project='hyperparameter-run',\n                 config=DBM_config)\nconfig = wandb.config  # We'll use this to configure our experiment\n"

### Initialize Mask R-CNN model for training

In [ ]:
'''
DBM_model = model.MaskRCNN(mode = 'training', 
                           model_dir = './', 
                           config = DBM_config)
'''

"\nDBM_model = model.MaskRCNN(mode = 'training', \n                           model_dir = './', \n                           config = DBM_config)\n"

### Load pre-trained weights from COCO data set (excluding last few layers)

In [ ]:
'''
DBM_model.load_weights(filepath = '/gdrive/My Drive/DBM Mask RCNN/mask_rcnn_coco.h5', 
                       by_name = True, 
                       exclude = ["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
'''

'\nDBM_model.load_weights(filepath = \'/gdrive/My Drive/DBM Mask RCNN/mask_rcnn_coco.h5\', \n                       by_name = True, \n                       exclude = ["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])\n'

### Creating an inference model to calculate mAP during training. Creating 2 callbacks, one for calculating mAP during training, other one to display metrics on Weights & Biases website

In [ ]:
'''
model_inference = model.MaskRCNN(mode = "inference", config = DBM_config, model_dir = './')

mAPCallback = model.MeanAveragePrecisionCallback(DBM_model, model_inference, valid_dataset, calculate_map_at_every_X_epoch=5, verbose=1)

callbacks = [mAPCallback, 
             WandbCallback()]
'''

'\nmodel_inference = model.MaskRCNN(mode = "inference", config = DBM_config, model_dir = \'./\')\n\nmAPCallback = model.MeanAveragePrecisionCallback(DBM_model, model_inference, valid_dataset, calculate_map_at_every_X_epoch=5, verbose=1)\n\ncallbacks = [mAPCallback, \n             WandbCallback()]\n'

In [ ]:
'''
model_inference = model.MaskRCNN(mode = "inference", config = DBM_config, model_dir = './')

mAPCallback = model.MeanAveragePrecisionCallback(DBM_model, model_inference, valid_dataset, calculate_map_at_every_X_epoch=5, verbose=1)
'''

'\nmodel_inference = model.MaskRCNN(mode = "inference", config = DBM_config, model_dir = \'./\')\n\nmAPCallback = model.MeanAveragePrecisionCallback(DBM_model, model_inference, valid_dataset, calculate_map_at_every_X_epoch=5, verbose=1)\n'

In [ ]:
'''
model_inference = model.MaskRCNN(mode = "inference", config = DBM_config, model_dir = './')

mAPCallback = model.MeanAveragePrecisionCallback(DBM_model, model_inference, valid_dataset, calculate_map_at_every_X_epoch=5, verbose=1)

callbacks = [mAPCallback]
'''

'\nmodel_inference = model.MaskRCNN(mode = "inference", config = DBM_config, model_dir = \'./\')\n\nmAPCallback = model.MeanAveragePrecisionCallback(DBM_model, model_inference, valid_dataset, calculate_map_at_every_X_epoch=5, verbose=1)\n\ncallbacks = [mAPCallback]\n'

### Training the model

In [ ]:
'''
DBM_model.train(train_dataset = train_set, 
                val_dataset = valid_dataset, 
                learning_rate = DBM_config.LEARNING_RATE, 
                epochs = 5, 
                layers = 'heads', 
                custom_callbacks = callbacks)
'''

"\nDBM_model.train(train_dataset = train_set, \n                val_dataset = valid_dataset, \n                learning_rate = DBM_config.LEARNING_RATE, \n                epochs = 5, \n                layers = 'heads', \n                custom_callbacks = callbacks)\n"

In [ ]:
def train():

  config_defaults = {
      'learning_rate': 0.001,
      'epochs': 10
  }

  # Initialize wandb with a sample project name
  wandb.init(config=config_defaults)

  # Iniialize model with hyperparameters
  keras.backend.clear_session()

  DBM_model = model.MaskRCNN(mode = 'training', 
                           model_dir = './', 
                           config = DBM_config)
  
  DBM_model.load_weights(filepath = '/gdrive/My Drive/DBM Mask RCNN/mask_rcnn_coco.h5', 
                       by_name = True, 
                       exclude = ["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
  
  model_inference = model.MaskRCNN(mode = "inference", config = DBM_config, model_dir = './')
  mAPCallback = model.MeanAveragePrecisionCallback(DBM_model, model_inference, valid_dataset, calculate_map_at_every_X_epoch=5, verbose=1)

  DBM_model.train(train_dataset = train_set, 
                  val_dataset = valid_dataset, 
                  learning_rate = wandb.config.learning_rate, 
                  epochs = wandb.config.epochs, 
                  layers = 'heads', 
                  custom_callbacks = [mAPCallback, WandbCallback()])

#train()

In [ ]:
sweep_config = {
    'method': 'random'
    }

In [ ]:
parameters_dict = {
    'STEPS_PER_EPOCH': {
        'values': [20, 40, 60]
        },
    'learning_rate': {
        'values': [0.005, 0.01, 0.0025]
        },
    'epochs': {
          'values': [13, 19, 23]
        },
    }

sweep_config['parameters'] = parameters_dict

In [ ]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'parameters': {'STEPS_PER_EPOCH': {'values': [30, 40, 50]},
                'epochs': {'values': [15, 20, 25]},
                'learning_rate': {'values': [0.001, 0.0008, 0.003]}}}


In [ ]:
wandb.agent("ronithsaju/desktop-sweeps/7ul5kwhu", train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: br0rrygb with config:
wandb: 	STEPS_PER_EPOCH: 50
wandb: 	epochs: 25
wandb: 	learning_rate: 0.001



Starting at epoch 0. LR=0.001

Checkpoint Path: ./dbm_cfg20210427T0020/mask_rcnn_dbm_cfg_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistr

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multi

Epoch 1/25
20/20 [==============================] - 92s 5s/step - loss: 3.2181 - rpn_class_loss: 0.4947 - rpn_bbox_loss: 0.5372 - mrcnn_class_loss: 0.3263 - mrcnn_bbox_loss: 0.8449 - mrcnn_mask_loss: 1.0150 - val_loss: 2.4120 - val_rpn_class_loss: 0.2358 - val_rpn_bbox_loss: 0.5293 - val_mrcnn_class_loss: 0.2730 - val_mrcnn_bbox_loss: 0.6645 - val_mrcnn_mask_loss: 0.7094


wandb: ERROR Can't save model, h5py returned error: can't pickle _thread.RLock objects


...

Epoch 2/25
19/20 [===========================>..] - ETA: 0s - loss: 1.9431 - rpn_class_loss: 0.1995 - rpn_bbox_loss: 0.4479 - mrcnn_class_loss: 0.4051 - mrcnn_bbox_loss: 0.4842 - mrcnn_mask_loss: 0.4064

/usr/local/lib/python3.7/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


20/20 [==============================] - 55s 3s/step - loss: 1.9334 - rpn_class_loss: 0.1944 - rpn_bbox_loss: 0.4764 - mrcnn_class_loss: 0.3870 - mrcnn_bbox_loss: 0.4690 - mrcnn_mask_loss: 0.4066 - val_loss: 1.9138 - val_rpn_class_loss: 0.1417 - val_rpn_bbox_loss: 0.4228 - val_mrcnn_class_loss: 0.4600 - val_mrcnn_bbox_loss: 0.4691 - val_mrcnn_mask_loss: 0.4202
Epoch 3/25
20/20 [==============================] - 26s 1s/step - loss: 1.6372 - rpn_class_loss: 0.1432 - rpn_bbox_loss: 0.3406 - mrcnn_class_loss: 0.2835 - mrcnn_bbox_loss: 0.4884 - mrcnn_mask_loss: 0.3814 - val_loss: 2.0186 - val_rpn_class_loss: 0.1321 - val_rpn_bbox_loss: 0.4625 - val_mrcnn_class_loss: 0.4813 - val_mrcnn_bbox_loss: 0.5737 - val_mrcnn_mask_loss: 0.3689
Epoch 4/25
20/20 [==============================] - 147s 7s/step - loss: 1.6365 - rpn_class_loss: 0.0899 - rpn_bbox_loss: 0.3210 - mrcnn_class_loss: 0.2896 - mrcnn_bbox_loss: 0.5321 - mrcnn_mask_loss: 0.4040 - val_loss: 2.1156 - val_rpn_class_loss: 0.2061 - val_r

/usr/local/lib/python3.7/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


20/20 [==============================] - 83s 4s/step - loss: 1.6720 - rpn_class_loss: 0.2095 - rpn_bbox_loss: 0.4322 - mrcnn_class_loss: 0.1985 - mrcnn_bbox_loss: 0.4946 - mrcnn_mask_loss: 0.3373 - val_loss: 1.6398 - val_rpn_class_loss: 0.1383 - val_rpn_bbox_loss: 0.4875 - val_mrcnn_class_loss: 0.3125 - val_mrcnn_bbox_loss: 0.3804 - val_mrcnn_mask_loss: 0.3211
Epoch 7/25
19/20 [===========================>..] - ETA: 0s - loss: 1.6691 - rpn_class_loss: 0.1462 - rpn_bbox_loss: 0.4462 - mrcnn_class_loss: 0.2429 - mrcnn_bbox_loss: 0.4643 - mrcnn_mask_loss: 0.3694

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="desktop-sweeps")

NameError: ignored

In [ ]:
wandb.agent(sweep_id, train)

SyntaxError: ignored

### Saving the trained weights generated

In [ ]:
model_path = 'DBM_mask_rcnn.h5'
DBM_model.keras_model.save_weights(model_path)

### Loading the model

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

#Loading the model in the inference mode
DBM_model = model.MaskRCNN(mode = "inference", config = DBM_config, model_dir = './')
# loading the trained weights o the custom dataset
DBM_model.load_weights(model_path, by_name=True)

### Displaying the results and saving it into DBM Model Results folder (inside DBM Mask RCNN folder)

In [ ]:
image_id = 0
image, image_meta, gt_class_id, gt_bbox, gt_mask = model.load_image_gt(valid_dataset, DBM_config, image_id, use_mini_mask=False)
info = valid_dataset.image_info[image_id]
#print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                      #valid_dataset.image_reference(image_id)))
# Run object detection
results = DBM_model.detect([image], verbose=1)

# Display results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            valid_dataset.class_names, r['scores'], 
                            title=str(info["id"]), show_mask = False)

### Saving all the results into DBM Model Results folder (inside DBM Mask RCNN folder)

In [ ]:
'''
APs = []
while image_id < 80:
  image, image_meta, gt_class_id, gt_bbox, gt_mask = model.load_image_gt(valid_dataset, DBM_config, image_id, use_mini_mask=False)
  info = valid_dataset.image_info[image_id]
  #print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                        #valid_dataset.image_reference(image_id)))
  # Run object detection
  results = DBM_model.detect([image], verbose=1)

  # Display results
  r = results[0]
  visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                              valid_dataset.class_names, r['scores'], 
                              title=str(info["id"]), show_mask = False)

  AP, precisions, recalls, overlaps =\
      utils.compute_ap(gt_bbox, gt_class_id, gt_mask, r['rois'], r['class_ids'], r['scores'], r['masks'], iou_threshold=0.5)

  APs.append(AP)
  print("AP: ", AP)
  print("precisions: ", precisions)
  print("recalls: ", recalls)
  print("overlaps: ", overlaps)
  print("APs: ", APs)
  print("mAP: ", np.mean(APs))
  utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask, r['rois'], r['class_ids'], r['scores'], r['masks'], iou_thresholds=None, verbose=1)

  image_id += 1
'''